In [1]:
import os
from dateutil.parser import parse as dateutil_parse
import time
import pandas as pd
import numpy as np
import cPickle

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.cross_validation import train_test_split,cross_val_score

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

from mylib import myStandardScaler,process_order,process_traffic,get_order_group,get_traffic_group,XY_order_traffic, prediction2submit, Search_Model, DISTRICTS

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order = pd.read_pickle(CLEAN_PATH+'train_order.pickle')
test_order = pd.read_pickle(CLEAN_PATH+'test_order.pickle')
train_traffic = pd.read_pickle(CLEAN_PATH+'train_traffic.pickle')
test_traffic = pd.read_pickle(CLEAN_PATH+'test_traffic.pickle')
train_weather = pd.read_pickle(CLEAN_PATH+'train_weather.pickle')
test_weather = pd.read_pickle(CLEAN_PATH+'test_weather.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

IOError: [Errno 2] No such file or directory: 'season_1/clean/train_order.pickle'

In [3]:
now = time.time()

train_order = process_order(train_order)
test_order = process_order(test_order)
train_traffic = process_traffic(train_traffic)
test_traffic = process_traffic(test_traffic)

train_order_group = get_order_group(train_order)
test_order_group = get_order_group(test_order)
train_traffic_group = get_traffic_group(train_traffic)
test_traffic_group = get_traffic_group(test_traffic)

train_slot = pd.Index(sorted(train_order['datetimeslot'].unique()))
train_slot = pd.Index(filter(lambda x: x%1000 >4,train_slot))
test_slot = test_target['datetimeslot']

stop = time.time()
print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Take 00:20:16


In [31]:
datetime1_index = pd.Series(range(len(train_order)),index=train_order['datetimeslot'])
datetime1_index

datetimeslot
20160101082          0
20160101059          1
20160101111          2
20160101134          3
20160101103          4
20160101038          5
20160101106          6
20160101074          7
20160101087          8
20160101132          9
20160101006         10
20160101079         11
20160101097         12
20160101118         13
20160101003         14
20160101103         15
20160101082         16
20160101127         17
20160101030         18
20160101126         19
20160101120         20
20160101098         21
20160101002         22
20160101044         23
20160101070         24
20160101070         25
20160101056         26
20160101093         27
20160101074         28
20160101074         29
                ...   
20160121103    8540584
20160121090    8540585
20160121049    8540586
20160121097    8540587
20160121085    8540588
20160121058    8540589
20160121055    8540590
20160121055    8540591
20160121065    8540592
20160121049    8540593
20160121111    8540594
20160121138    854059

In [30]:
train_order.ix[datetime1_index.ix['20160101001':'20160101144']].sort_values(by = 'Time')

KeyError: '20160101001'

In [12]:
cPickle.dump(train_order_group,open(CLEAN_PATH+'train_order_group.pickle','wb')) 
cPickle.dump(test_order_group,open(CLEAN_PATH+'test_order_group.pickle','wb')) 
cPickle.dump(train_traffic_group,open(CLEAN_PATH+'train_traffic_group.pickle','wb')) 
cPickle.dump(test_traffic_group,open(CLEAN_PATH+'test_traffic_group.pickle','wb')) 

In [4]:
now = time.time()

train_XY_group = dict()
for district in DISTRICTS:
    train_XY_group[district] = XY_order_traffic(district,train_order_group,train_traffic_group,train_slot)
test_XY_group = dict()
for district in DISTRICTS:
    test_XY_group[district] = XY_order_traffic(district,test_order_group,test_traffic_group,test_slot)
for district in DISTRICTS:
    scaler = myStandardScaler()
    train_XY_group[district][0] = scaler.fit_transform(train_XY_group[district][0])
    test_XY_group[district][0] = scaler.transform(test_XY_group[district][0])
    
stop = time.time()
print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Take 00:00:16


In [28]:
the_scorer = make_scorer(mymetrics,greater_is_better=False)

In [30]:
params = {'n_estimators': 111, 'max_depth':12}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.83046714 -0.81149442 -0.87328421]
0.838415255249
0.0258440710689


In [29]:
params = {'n_estimators': 111, 'max_depth':8}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.83892389 -0.85266232 -0.86648345]
0.852689886364
0.0112511601004


In [31]:
params = {'n_estimators': 111, 'max_depth':14}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.83138623 -0.79353353 -0.87784487]
0.834254877388
0.0344796778876


In [33]:
params = {'n_estimators': 111, 'max_depth':16}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.83754856 -0.79839981 -0.86636178]
0.834103383561
0.0278521018729


In [37]:
params = {'n_estimators': 111, 'max_depth':20}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.84747325 -0.79683217 -0.88360061]
0.842635345673
0.035587866549


In [38]:
params = {'n_estimators': 111, 'max_depth':None}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(ExtraTreesRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.84091984 -0.82700419 -0.90147296]
0.856465662879
0.0323280491496


In [41]:
params = {'n_estimators': 111, 'max_depth':15}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(ExtraTreesRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.91380375 -0.80811523 -0.83499188]
0.852303616713
0.0448500318359


In [42]:
params = {'n_estimators': 111, 'max_depth':20}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(ExtraTreesRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.91519035 -0.78509206 -0.83041531]
0.843565908188
0.0539202807956


In [43]:
params = {'n_estimators': 111, 'min_samples_split':4}
trainX, trainY = train_XY_group.values()[0]
scores = cross_val_score(ExtraTreesRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.90858172 -0.82475439 -0.84563716]
0.859657760219
0.0356294646107


In [48]:
params = {'n_estimators': 111, 'max_depth':16}
trainX, trainY = train_XY_group.values()[7]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

[-0.56030667 -0.5075259  -0.56756581]
0.545132796257
0.0267567128962


In [47]:
params = {'n_estimators': 111, 'max_depth':16}
trainX, trainY = train_XY_group.values()[7]
scores = cross_val_score(RandomForestRegressor(**params), trainX, trainY, scoring=the_scorer)
print scores
print -np.mean(scores)
print np.std(scores)

20160101005     1.0
20160101006     2.0
20160101007     0.0
20160101008     0.0
20160101009     1.0
20160101010     1.0
20160101011     0.0
20160101012     0.0
20160101013     1.0
20160101014     1.0
20160101015     1.0
20160101016     7.0
20160101017     5.0
20160101018     8.0
20160101019     2.0
20160101020     4.0
20160101021     7.0
20160101022     6.0
20160101023     0.0
20160101024     3.0
20160101025    14.0
20160101026     8.0
20160101027     6.0
20160101028     0.0
20160101029     2.0
20160101030     4.0
20160101031     4.0
20160101032     1.0
20160101033     0.0
20160101034     2.0
               ... 
20160121115     1.0
20160121116     1.0
20160121117     0.0
20160121118     1.0
20160121119     2.0
20160121120     2.0
20160121121     1.0
20160121122     0.0
20160121123     3.0
20160121124     3.0
20160121125     4.0
20160121126     5.0
20160121127     3.0
20160121128     1.0
20160121129     2.0
20160121130     6.0
20160121131     2.0
20160121132     1.0
20160121133     2.0


In [ ]:
grid_params = {'n_estimators': [120] ,'max_depth': np.arange(10, 18), 'min_samples_leaf': [2, 6, 10], 
                     'min_samples_split': [2, 6, 10], 'max_features': ['log2', 'sqrt', None]}
trainX, trainY = train_XY_group.values()[7]
rfr_searcher = Search_Model(RandomForestRegressor)
rfr_searcher.fit(grid_params, trainX, trainY)
rfr_searcher.predict(trainX)

In [ ]:
# grid_params = {'n_estimators': [150] ,'max_depth': np.arange(8, 18), 'min_samples_leaf': [2, 6, 10], 
#                      'min_samples_split': [2, 6, 10], 'bootstrap': [True, False], 'max_features': ['log2', 'sqrt', None]}
# search_models = {district: Search_Model(RandomForestRegressor) for district in DISTRICTS}
# test_prediction = dict()
# for district, model in search_models.items():
#     now = time.time()
#     print 'Searching %s...'%district
#     model.fit(grid_params,*train_XY_group[district])
# #     train_prediction = pd.Series(np.floor(model.predict(train_XY_group[district][0])),index=train_XY_group[district][1].index)
#     test_prediction[district] = model.predict(test_XY_group[district][0]) - test_XY_group[district][1].fillna(0)
# #     print 'r2 score', model.score(*train_XY_group[district])
# #     print distri'metric', metrics(model.predict(train_XY_group[district][0]),train_prediction)
#     stop = time.time()
#     print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)